## Initial Settings

In [1]:
# Load Required Libraries
import os
import sys
import platform
import pandas as pd
import numpy as np
from plydata import *
import geopandas as gpd
import json

from bokeh.io import show, output_file, curdoc, output_notebook
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, Slider, HoverTool
from bokeh.palettes import brewer
from bokeh.layouts import widgetbox, row, column



# Set Project Path
OS = platform.system()

if OS == "Linux":
    PROJECT_PATH = "/home/pooya/work/HydroTech/"
else:
    PROJECT_PATH = "c:/work/HydroTech/"



# Load Required Functions
# Function 01 - Extract 'Region', 'District' from 'Peyman' Column
def extractRD(x, para):

    if para == 'ناحیه':
        if para not in x:
            return np.nan
        elif (x.index(para) + 1) >= len(x):
            return np.nan
        else:
            return str(x[x.index(para) + 1]).zfill(2)

    if para == 'منطقه':
        if para not in x:
            if ('کمربند' in x) and ('جنوبی' in x):
                return '14'
            elif ('کمربند' in x) and ('شمالی' in x):
                return '15'
            elif ('سازمان' in x) and ('پارک‌ها' in x):
                return '16'
            else:
                return np.nan
        elif (x.index(para) + 1) >= len(x):
            return np.nan
        else:
            if str(x[x.index(para) + 1]) == 'ثامن':
                return '13'
            else:
                return str(x[x.index(para) + 1]).zfill(2)

In [2]:
#  Load Data: Read Data
raw_data = pd.read_excel(PROJECT_PATH + 'Data/Processed_Data/Merged_Data.xlsx')


#  Data Cleansing: Remove Duplicated Rows
# 01. Report Duplicated Rows
tmp = raw_data[
    raw_data.duplicated(
        subset=list(raw_data.columns)[1:],
        keep=False
    )
]

tmp = tmp.sort_values(['پیمان', 'نام لکه', 'نوع قلم', 'زیرمجموعه هر قلم'])

tmp.to_excel(PROJECT_PATH + "Report/Duplicate_Rows.xlsx", index=False)

print(f"Total Number of Duplicate Rows in a Data: {tmp.shape[0]}")

tmp = raw_data[
    raw_data.duplicated(
        subset=list(raw_data.columns)[1:],
        keep='first')
]

print(f"Total Number of Duplicate Items in a Data: {tmp.shape[0]}")

del tmp

# 02. Remove Duplicated Rows
raw_data = raw_data.drop_duplicates(
    subset=list(raw_data.columns)[1:],
    keep='first'
)

print(f"Data Size: {raw_data.shape}")


#  Data Cleansing: Remove Some Rows
tmp = raw_data["نوع آیتم"] == "حجمی"
tmp = tmp[tmp.values == True]

raw_data.drop(
    index=tmp.index,
    inplace=True
)

del tmp

print(f"Data Size: {raw_data.shape}")


#  Data Cleansing: Extract Region And District
tmp = raw_data["پیمان"].str.strip().str.split()

raw_data["Region"] = tmp.apply(extractRD, para="منطقه")

raw_data["District"] = tmp.apply(extractRD, para="ناحیه")

del tmp


#  Data Cleansing: Extract Peyman
# Extract Uniqe Peyman
tmp = raw_data.groupby(['Region', 'District'])['پیمان']
tmp = tmp.value_counts(dropna=False, sort=False)
tmp = pd.DataFrame(tmp)
tmp = tmp.rename(columns={'پیمان': 'Count'}).reset_index()

Peyman = []
for R in list(tmp['Region'].unique()):
    tmpR = tmp[tmp['Region'] == R]
    for D in list(tmpR['District'].unique()):
        tmpD = tmpR[tmpR['District'] == D]
        Peyman += list(range(1, len(tmpD) + 1))

tmp['Peyman'] = Peyman
tmp['Peyman'] = tmp['Peyman'].astype(str).str.zfill(2)

# Add Peyman To raw_data
tmp = tmp >> select('Region', 'District', 'پیمان', 'Peyman')

raw_data = pd.merge(raw_data,
                    tmp,
                    how='left',
                    on=['Region', 'District', 'پیمان'])

del tmp, tmpR, tmpD


#  Data Cleansing: Extract Address
# Extract Uniqe Address
tmp = raw_data.groupby(['Region', 'District', 'Peyman'])['نام لکه']
tmp = tmp.value_counts(dropna=False, sort=False)
tmp = pd.DataFrame(tmp)
tmp = tmp.rename(columns={'نام لکه': 'Count'}).reset_index()

Address = []
for R in list(tmp['Region'].unique()):
    tmpR = tmp[tmp['Region'] == R]
    for D in list(tmpR['District'].unique()):
        tmpD = tmpR[tmpR['District'] == D]
        for P in list(tmpD['Peyman'].unique()):
            tmpP = tmpD[tmpD['Peyman'] == P]
            Address += list(range(1, len(tmpP) + 1))

tmp['Address'] = Address
tmp['Address'] = tmp['Address'].astype(str).str.zfill(3)

# Add Address To raw_data
tmp = tmp >> select('Region', 'District', 'Peyman', 'نام لکه', 'Address')

raw_data = pd.merge(raw_data,
                    tmp,
                    how='left',
                    on=['Region', 'District', 'Peyman', 'نام لکه'])

del tmp, tmpR, tmpD, tmpP


#  Report: Check Region
raw_data['Region'].value_counts(dropna=False, sort=True)


#  Report: Check District
raw_data['District'].value_counts(dropna=False, sort=True)


#  Report: Check Peyman
raw_data['Peyman'].value_counts(dropna=False)


#  Report: Check Address
raw_data['Address'].value_counts(dropna=False)


#  Data Cleansing: Define Spots Class
Spots_Class = {
    'میادین': '01',
    'لچکی ها': '02',
    'آیلند های بزرگراه': '03',
    'آیلند ها': '04',
    'حاشیه های بزرگراه': '05',
    'حاشیه معابر': '06',
    'بوستان خطی': '07',
    'پارک های زیر 6 هکتار': '08',
    'پارک های بین 6 تا 10 هکتار': '09',
    'پارک های بالای 10 هکتار': '10',
    'جنگل کاری داخل محدوده': '11',
    'کمربندی': '12',
    'کمربند سبز حفاظتی': '13',
}


#  Data Cleansing: Spots Class
# Extract All Spots Class From Data
tmp = raw_data['نوع لکه'].value_counts(dropna=False, sort=True)
tmp = tmp.reset_index().rename(columns={'index': 'نوع لکه',
                                        'نوع لکه': 'تعداد'})

tmp['Spot'] = list(map(Spots_Class.get, tmp['نوع لکه']))

file_name = PROJECT_PATH + 'Report/Type_of_Spots.xlsx'
tmp.to_excel(file_name, index=False)

# Add Spots Class to raw_data
tmp = tmp[['نوع لکه', 'Spot']]
raw_data = pd.merge(raw_data,
                    tmp,
                    how='left',
                    on=['نوع لکه'])

del tmp, file_name


#  Data Cleansing: Define Irrigation Method Class
Irrigation_Method_Class = {
    'آبیاری ثقلی': '01',
    'آبیاری تانکری': '02',
    'آبیاری شلنگی': '03',
    'آبیاری تحت فشار': '04'
}


#  Data Cleansing: Irrigation Method Class
# Extract All Irrigation Method Class From Data
tmp = raw_data['زیرمجموعه هر قلم'].value_counts(dropna=False, sort=True)
tmp = tmp.reset_index().rename(columns={'index': 'زیرمجموعه هر قلم',
                                        'زیرمجموعه هر قلم': 'تعداد'})

tmp['Irrigation'] = list(map(Irrigation_Method_Class.get,
                             tmp['زیرمجموعه هر قلم']))

# Add Irrigation Method Class to raw_data
tmp = tmp[['زیرمجموعه هر قلم', 'Irrigation']]
raw_data = pd.merge(raw_data,
                    tmp,
                    how='left',
                    on=['زیرمجموعه هر قلم'])

raw_data.fillna(value=np.nan, inplace=True)

del tmp


#  Data Cleansing: Define Species Plant Class
Species_Plant_Class = {
    'چمن': '01',
    'گل دائم باغچه های معمولی': '02',
    'گل دائم فلاورباکسهای سطوح شیب دار': '04',
    'گل فصل باغچه های معمولی': '05',
    'پرچین': '06',
    'درخت و درختچه': '10',
    'درختان جنگلی': '12',
    'گل فصل فلاورباکس های سطوح شیب دار': '14'
}

#  Data Cleansing: Species Plant Class
# Extract All Species Plant Class From Data
tmp = raw_data['نوع قلم'].value_counts(dropna=False, sort=True)
tmp = tmp.reset_index().rename(columns={'index': 'نوع قلم',
                                        'نوع قلم': 'تعداد'})

tmp['Species'] = list(map(Species_Plant_Class.get, tmp['نوع قلم']))

# Add Species Plant Class to raw_data
tmp = tmp[['نوع قلم', 'Species']]
raw_data = pd.merge(raw_data,
                    tmp,
                    how='left',
                    on=['نوع قلم'])
raw_data.fillna(value=np.nan, inplace=True)

del tmp


#  Data Cleansing: Generate ID
# Check Number of NaN
raw_data[['Region', 'District', 'Peyman',
          'Address', 'Spot', 'Irrigation', 'Species']].isnull().sum()

# Generate ID
tmp = 'Region + District + Peyman + Address + "-" + Spot + Irrigation + Species'
raw_data = raw_data >> define(ID=tmp)

del tmp

#  Report: ID Check
tmp = raw_data.astype(str).groupby(['ID']).size()
tmp = pd.DataFrame(tmp).rename(columns={0: 'Count'})
tmp >> query("Count >= 2")

del tmp

#  Report: Save And Remove Duplicate ID
tmp = raw_data.dropna(subset=['ID']).duplicated(subset="ID", keep=False)
tmp = raw_data.dropna(subset=['ID'])[tmp.values]

file_name = PROJECT_PATH + '/Report/Duplicate_ID.xlsx'
tmp.to_excel(file_name, index=False)

tmp = raw_data.dropna(subset=['ID']).duplicated(subset="ID", keep='last')
tmp = tmp[tmp.values == True].index
raw_data.drop(axis=0, index=tmp, inplace=True)

del tmp, file_name

print(f"Data Size: {raw_data.shape}")


#  Data Cleansing: Change Columns Name
data = raw_data >> select('ID', 'Region', 'District', 'Peyman', 'Address',
                          'Spot', 'Irrigation', 'Species', 'نمایش آخرین ریزمتره (ریزمتره نهایی)',
                          'مساحت لکه (مترمربع)', 'مساحت پیمان (مترمربع)', 'نوع آیتم', 'نوع لکه',
                          'نام لکه', 'نوع قلم', 'زیرمجموعه هر قلم', 'پیمان', 'ردیف')

data.rename(columns={
    'نمایش آخرین ریزمتره (ریزمتره نهایی)': 'Irrigation_Area',
    'مساحت لکه (مترمربع)': 'Address_Area',
    'مساحت پیمان (مترمربع)': 'Peyman_Area',
    'نوع آیتم': 'Item_Type',
    'نوع لکه': 'Lake_Type',
    'نام لکه': 'Lake_Name',
    'نوع قلم': 'Ghalam_Type',
    'زیرمجموعه هر قلم': 'Ghalam_SubType',
    'پیمان': 'Check_point',
    'ردیف': 'Radif'
}, inplace=True)


#  Data Cleansing: Remove 'meter' From Some Rows
tmp = data >> query(
    'Ghalam_Type == "درخت و درختچه" and Ghalam_SubType == "متر"'
)

file_name = PROJECT_PATH + "Report/Error_Metr.xlsx"
tmp.to_excel(file_name, index=False)

data.loc[list(tmp.index), "Ghalam_SubType"] = "متر مربع"

del tmp, file_name


#  Add Some Element To Database
# Define Extera Class
Extera_Class = {
    'متر مربع': '01',
    'اصله': '02'
}

# Extract All Extera Class In Data
tmp = data['Ghalam_SubType'].value_counts(dropna=False, sort=True)
tmp = tmp.reset_index().rename(columns={'index': 'Ghalam_SubType',
                                        'Ghalam_SubType': 'Count'})
tmp['Extera'] = list(map(Extera_Class.get, tmp['Ghalam_SubType']))

# Add Extera Class to data
tmp = tmp[['Ghalam_SubType', 'Extera']]

data = pd.merge(data,
                tmp,
                how='left',
                on=['Ghalam_SubType'])

data.fillna(value=np.nan, inplace=True)

# Remove From Irrigation_Area
data = data >> define(Tree=if_else('Extera == "02"',
                                   'Irrigation_Area',
                                   np.nan))

data = data >> define(M2=if_else('Extera == "01"',
                                 'Irrigation_Area',
                                 np.nan))

data = data >> define(Irrigation_Area=if_else('Extera == "02"',
                                              np.nan,
                                              'Irrigation_Area'))

data = data >> define(Irrigation_Area=if_else('Extera == "01"',
                                              np.nan,
                                              'Irrigation_Area'))

#  Modified ID
data = data >> define(Species=if_else('Extera == "02"', '"13"', 'Species'))

s = 'Region + District + Peyman + Address + "-" + Spot + "00" + Species'
data = data >> define(ID=if_else('Extera == "02"', s, 'ID'))

data['Species'].replace(to_replace='nan', value=np.NaN, inplace=True)
data['ID'].replace(to_replace='nan', value=np.NaN, inplace=True)

data = data >> call('.dropna', subset=['ID'])


#  Report: Check Region, District and Peyman
tmpG = data.astype(str).groupby(
    ['Region', 'District', 'Peyman', 'Check_point'])
tmp = pd.DataFrame(tmpG.size())
tmp = tmp.reset_index()
tmp = tmp.rename(columns={'Region': 'منطقه',
                          'District': 'ناحیه',
                          'Peyman': 'پیمان',
                          'Check_point': 'نام پیمان',
                          0: 'تعداد ردیف'})

file_name = PROJECT_PATH + "Report/Region_District_Peyman.xlsx"
tmp.to_excel(file_name, index=False)

print(tmp)
del tmpG, tmp, file_name

#  Report: Check Region, District, Peyman, Address
tmpG = data.astype(str).groupby(
    ['Region', 'District', 'Peyman', 'Address', 'Lake_Name']
)
tmp = pd.DataFrame(tmpG.size())
tmp = tmp.reset_index()
tmp = tmp.rename(columns={'Region': 'منطقه',
                          'District': 'ناحیه',
                          'Peyman': 'پیمان',
                          'Address': 'لکه',
                          'Lake_Name': 'نام لکه',
                          0: 'تعداد ردیف'})

file_name = PROJECT_PATH + "Report/Region_District_Peyman_Address.xlsx"
tmp.to_excel(file_name, index=False)

print(tmp)
del tmpG, tmp, file_name


#  Report: Species Plant Class In Data
tmp = data.groupby(['Ghalam_Type', 'Ghalam_SubType']).size()
tmp = tmp.reset_index()
tmp = tmp.rename(columns={'Ghalam_Type': 'نوع قلم',
                          'Ghalam_SubType': 'زیرمجموعه هر قلم',
                          0: 'تعداد ردیف'})

file_name = PROJECT_PATH + '/Report/Ghalam_SubGhalam.xlsx'
tmp.to_excel(file_name, index=False)

del tmp, file_name

tmp = data.groupby(['Ghalam_SubType', 'Ghalam_Type']).size()
tmp = tmp.reset_index()
tmp = tmp.rename(columns={'Ghalam_Type': 'نوع قلم',
                          'Ghalam_SubType': 'زیرمجموعه هر قلم',
                          0: 'تعداد ردیف'})

file_name = PROJECT_PATH + '/Report/SubGhalam_Ghalam.xlsx'
tmp.to_excel(file_name, index=False)

del tmp, file_name


#  Report: Irrigation - Species - RDPA
tmpG = data.groupby(
    ['Region', 'District', 'Peyman', 'Address', 'Irrigation', 'Species']
)

tmp = tmpG.agg({
    'Irrigation_Area': 'sum',
    'Address_Area': 'mean',
    'Peyman_Area': 'mean'
})

file_name = PROJECT_PATH + "Report/Report_RDPAIS_Irrigated_Address_Peyman_Area.xlsx"

tmp.to_excel(file_name, index=True)

del tmpG, tmp, file_name


#  Report: Irrigation - Species - RDP
tmpG = data.groupby(
    ['Region', 'District', 'Peyman', 'Irrigation', 'Species']
)

tmp = tmpG.agg({
    'Irrigation_Area': 'sum'
})

file_name = PROJECT_PATH + "Report/Report_RDPIS_Irrigated_Area.xlsx"

tmp.to_excel(file_name, index=True)

del tmpG, tmp, file_name


#  Report: Species - Irrigation - RDP
tmpG = data.groupby(
    ['Region', 'District', 'Peyman', 'Species', 'Irrigation']
)

tmp = tmpG.agg({
    'Irrigation_Area': 'sum'
})

file_name = PROJECT_PATH + "Report/Report_RDPSI_Irrigated_Area.xlsx"

tmp.to_excel(file_name, index=True)

del tmpG, tmp, file_name


#  Report: Irrigation - Species - RD
tmpG = data.groupby(
    ['Region', 'District', 'Irrigation', 'Species']
)

tmp = tmpG.agg({
    'Irrigation_Area': 'sum'
})

file_name = PROJECT_PATH + "Report/Report_RDIS_Irrigated_Area.xlsx"

tmp.to_excel(file_name, index=True)

del tmpG, tmp, file_name


#  Report:  Species - Irrigation - RD
tmpG = data.groupby(
    ['Region', 'District', 'Species', 'Irrigation']
)

tmp = tmpG.agg({
    'Irrigation_Area': 'sum'
})

file_name = PROJECT_PATH + "Report/Report_RDSI_Irrigated_Area.xlsx"

tmp.to_excel(file_name, index=True)

del tmpG, tmp, file_name


#  Report: Irrigation - Species - R
tmpG = data.groupby(
    ['Region', 'Irrigation', 'Species']
)

tmp = tmpG.agg({
    'Irrigation_Area': 'sum'
})

file_name = PROJECT_PATH + "Report/Report_RIS_Irrigated_Area.xlsx"

tmp.to_excel(file_name, index=True)

del tmpG, tmp, file_name


#  Report: Irrigation - Species - R
tmpG = data.groupby(
    ['Region', 'Species', 'Irrigation']
)

tmp = tmpG.agg({
    'Irrigation_Area': 'sum'
})

file_name = PROJECT_PATH + "Report/Report_RSI_Irrigated_Area.xlsx"

tmp.to_excel(file_name, index=True)

del tmpG, tmp, file_name


#  Report: Irrigation - Species
tmp = data.pivot_table(values="Irrigation_Area",
                       index=['Species'],
                       columns=['Irrigation'],
                       aggfunc=['count', 'sum'],
                       margins=True,
                       margins_name="Total")

file_name = PROJECT_PATH + "Report/PivotTable_Species_Irrigation_Count_Area.xlsx"
tmp.to_excel(file_name)


#  Report: Irrigation - Species - Region
tmp = data.pivot_table(values="Irrigation_Area",
                       index=['Region', 'Species'],
                       columns=['Irrigation'],
                       aggfunc=['count', 'sum'],
                       margins=True,
                       margins_name="Total")

file_name = PROJECT_PATH + "Report/PivotTable_Region_Species_Irrigation_Count_Area.xlsx"
tmp.to_excel(file_name)


#  Report: Irrigation - Species - Region - District
tmp = data.pivot_table(values="Irrigation_Area",
                       index=['Region', 'District', 'Species'],
                       columns=['Irrigation'],
                       aggfunc=['count', 'sum'],
                       margins=True,
                       margins_name="Total")

file_name = PROJECT_PATH + \
    "Report/PivotTable_Region_District_Species_Irrigation_Count_Area.xlsx"
tmp.to_excel(file_name)


#  Check Peyman Area
tmpG = data.groupby(['Region', 'District', 'Peyman'])

tmp = tmpG.agg({
    'Peyman_Area': ['min', 'max']
}).reset_index()

tmp.columns = ['Region', 'District', 'Peyman',
               'Peyman_Area_min', 'Peyman_Area_max']

tmp = tmp >> define(Check=if_else('Peyman_Area_min == Peyman_Area_max',
                                  True,
                                  False))

tmp.to_excel(PROJECT_PATH + "Report/Peyman_Area_Check.xlsx", index=False)


#  ETL Calculation - Efficacy Class
Eff = {
    '01': 0.70,
    '02': 0.60,
    '03': 0.80,
    '04': 0.80
}

data['Eff'] = list(map(Eff.get, data['Irrigation']))


#  ETL Calculation - Microclimate Class
Kmc = {
    '01': 1.20,
    '02': 1.20,
    '03': 1.15,
    '04': 1.10,
    '05': 1.10,
    '06': 1.05,
    '07': 1.05,
    '08': 1.00,
    '09': 0.90,
    '10': 0.80,
    '11': 1.00,
    '12': 1.05,
    '13': 1.05
}

data['Kmc'] = list(map(Kmc.get, data['Spot']))


#  ETL Calculation - Species Plant Class
Ksp = {
    '01': 0.70,
    '02': 0.75,
    '04': 0.80,
    '05': 0.80,
    '06': 0.80,
    '10': 0.70,
    '12': 0.49,
    '13': 0.00,
    '14': 0.80
}

data['Ksp'] = list(map(Ksp.get, data['Species']))


#  ETL Calculation - Change Species Plant Class In Year (1)
KspChangeYear = {
    '01': [0.60, 0.75, 0.90, 1.00, 0.95, 0.80, 0.60, 0.45, 0.35, 0.25, 0.30, 0.40],
    '02': [0.60, 0.75, 0.90, 1.00, 0.95, 0.80, 0.60, 0.45, 0.35, 0.25, 0.30, 0.40],
    '04': [0.60, 0.75, 0.90, 1.00, 0.95, 0.80, 0.60, 0.45, 0.35, 0.25, 0.30, 0.40],
    '05': [0.60, 0.75, 0.90, 1.00, 0.95, 0.80, 0.60, 0.45, 0.35, 0.25, 0.30, 0.40],
    '06': [0.60, 0.75, 0.90, 1.00, 0.95, 0.80, 0.60, 0.45, 0.35, 0.25, 0.30, 0.40],
    '10': [0.60, 0.75, 0.90, 1.00, 0.95, 0.80, 0.60, 0.45, 0.35, 0.25, 0.30, 0.40],
    '12': [0.60, 0.75, 0.90, 1.00, 0.95, 0.80, 0.60, 0.45, 0.35, 0.25, 0.30, 0.40],
    '13': [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
    '14': [0.60, 0.75, 0.90, 1.00, 0.95, 0.80, 0.60, 0.45, 0.35, 0.25, 0.30, 0.40]
}

Ksp_Month_Name = ['Ksp_M01', 'Ksp_M02', 'Ksp_M03', 'Ksp_M04', 'Ksp_M05', 'Ksp_M06',
                  'Ksp_M07', 'Ksp_M08', 'Ksp_M09', 'Ksp_M10', 'Ksp_M11', 'Ksp_M12']

KspChangeYear = pd.DataFrame(KspChangeYear, index=Ksp_Month_Name).T

KspChangeYear = KspChangeYear.reset_index()
KspChangeYear = KspChangeYear.rename(columns={'index': 'Species'})

data = data >> left_join(KspChangeYear, on='Species')


#  ETL Calculation - Change Species Plant Class In Year (2)
tmpG = data.groupby(by=['Region', 'District', 'Peyman',
                        'Address', 'Irrigation'])['Irrigation_Area']

tmp = tmpG.sum().reset_index().rename(
    columns={'Irrigation_Area': 'SubIrrigation_Area'}
)

data = data >> left_join(tmp,
                         on=['Region', 'District', 'Peyman', 'Address', 'Irrigation'])

data = data >> define(
    Ksp_M01='Ksp_M01 * Ksp * Irrigation_Area / SubIrrigation_Area')
data = data >> define(
    Ksp_M02='Ksp_M02 * Ksp * Irrigation_Area / SubIrrigation_Area')
data = data >> define(
    Ksp_M03='Ksp_M03 * Ksp * Irrigation_Area / SubIrrigation_Area')
data = data >> define(
    Ksp_M04='Ksp_M04 * Ksp * Irrigation_Area / SubIrrigation_Area')
data = data >> define(
    Ksp_M05='Ksp_M05 * Ksp * Irrigation_Area / SubIrrigation_Area')
data = data >> define(
    Ksp_M06='Ksp_M06 * Ksp * Irrigation_Area / SubIrrigation_Area')
data = data >> define(
    Ksp_M07='Ksp_M07 * Ksp * Irrigation_Area / SubIrrigation_Area')
data = data >> define(
    Ksp_M08='Ksp_M08 * Ksp * Irrigation_Area / SubIrrigation_Area')
data = data >> define(
    Ksp_M09='Ksp_M09 * Ksp * Irrigation_Area / SubIrrigation_Area')
data = data >> define(
    Ksp_M10='Ksp_M10 * Ksp * Irrigation_Area / SubIrrigation_Area')
data = data >> define(
    Ksp_M11='Ksp_M11 * Ksp * Irrigation_Area / SubIrrigation_Area')
data = data >> define(
    Ksp_M12='Ksp_M12 * Ksp * Irrigation_Area / SubIrrigation_Area')

tmpG = data.groupby(
    by=['Region', 'District', 'Peyman', 'Address', 'Irrigation'])

tmpG = tmpG['Ksp_M01', 'Ksp_M02', 'Ksp_M03', 'Ksp_M04', 'Ksp_M05', 'Ksp_M06',
            'Ksp_M07', 'Ksp_M08', 'Ksp_M09', 'Ksp_M10', 'Ksp_M11', 'Ksp_M12']

tmp = tmpG.sum().reset_index()

tmp = tmp.rename(columns={
    'Ksp_M01': 'Ksp_M01_Sum_SubIrigation',
    'Ksp_M02': 'Ksp_M02_Sum_SubIrigation',
    'Ksp_M03': 'Ksp_M03_Sum_SubIrigation',
    'Ksp_M04': 'Ksp_M04_Sum_SubIrigation',
    'Ksp_M05': 'Ksp_M05_Sum_SubIrigation',
    'Ksp_M06': 'Ksp_M06_Sum_SubIrigation',
    'Ksp_M07': 'Ksp_M07_Sum_SubIrigation',
    'Ksp_M08': 'Ksp_M08_Sum_SubIrigation',
    'Ksp_M09': 'Ksp_M09_Sum_SubIrigation',
    'Ksp_M10': 'Ksp_M10_Sum_SubIrigation',
    'Ksp_M11': 'Ksp_M11_Sum_SubIrigation',
    'Ksp_M12': 'Ksp_M12_Sum_SubIrigation'
})

data = data >> left_join(tmp,
                         on=['Region', 'District', 'Peyman', 'Address', 'Irrigation'])


#  ETL Calculation - Density Plant Class
Kd = {
    1: 1,
    2: 1,
    3: 1.2,
    4: 1.2,
    5: 1.3,
    6: 1.3,
    7: 1.3,
    8: 1.3
}


def name(x):
    x = list(x)
    if "10" in x and "13" in x:
        return -1
    else:
        return 0


tmpAG = data.groupby(
    by=['Region', 'District', 'Peyman', 'Address', 'Irrigation'])
tmpA = tmpAG.size().reset_index().rename(columns={0: 'Species_Count'})

tmpBG = data.groupby(by=['Region', 'District', 'Peyman', 'Address', 'Extera'])
tmpB = tmpBG.size().reset_index().rename(columns={0: 'Asleh'})

tmpC = tmpA >> left_join(tmpB, on=['Region', 'District', 'Peyman', 'Address'])
tmpC = tmpC >> call(pd.DataFrame.fillna, 0)

tmpDG = data.groupby(by=['Region', 'District', 'Peyman', 'Address'])
tmpD = tmpDG.agg({'Species': name})
tmpD = tmpD.reset_index().rename(columns={'Species': 'Double_Asleh'})

tmpE = tmpC >> left_join(tmpD, on=['Region', 'District', 'Peyman', 'Address'])
tmpE = tmpE >> define(Species_Count='Species_Count + Asleh + Double_Asleh')
tmpE = tmpE >> select('Extera', 'Asleh', 'Double_Asleh', drop=True)

data = data >> left_join(tmpE,
                         on=['Region', 'District', 'Peyman', 'Address', 'Irrigation'])

data['Kd'] = list(map(Kd.get, data['Species_Count']))


#  ETL Calculation - KL Calculate
data = data >> define(KL_M01='Ksp_M01_Sum_SubIrigation * Kmc * Kd')
data = data >> define(KL_M02='Ksp_M02_Sum_SubIrigation * Kmc * Kd')
data = data >> define(KL_M03='Ksp_M03_Sum_SubIrigation * Kmc * Kd')
data = data >> define(KL_M04='Ksp_M04_Sum_SubIrigation * Kmc * Kd')
data = data >> define(KL_M05='Ksp_M05_Sum_SubIrigation * Kmc * Kd')
data = data >> define(KL_M06='Ksp_M06_Sum_SubIrigation * Kmc * Kd')
data = data >> define(KL_M07='Ksp_M07_Sum_SubIrigation * Kmc * Kd')
data = data >> define(KL_M08='Ksp_M08_Sum_SubIrigation * Kmc * Kd')
data = data >> define(KL_M09='Ksp_M09_Sum_SubIrigation * Kmc * Kd')
data = data >> define(KL_M10='Ksp_M10_Sum_SubIrigation * Kmc * Kd')
data = data >> define(KL_M11='Ksp_M11_Sum_SubIrigation * Kmc * Kd')
data = data >> define(KL_M12='Ksp_M12_Sum_SubIrigation * Kmc * Kd')

Total Number of Duplicate Rows in a Data: 5421
Total Number of Duplicate Items in a Data: 2720
Data Size: (58745, 15)
Data Size: (40999, 15)
Data Size: (40910, 23)
   منطقه ناحیه پیمان                   نام پیمان  تعداد ردیف
0     01    01    01         الف ناحیه 1 منطقه 1         275
1     01    02    01  الف ناحیه 2 منطقه 1 - 9808         258
2     01    02    02    ب ناحیه 2 منطقه 1 - 9808         243
3     01    03    01      ناحیه 3 منطقه 1 - 9711         229
4     02    01    01  الف ناحیه 1 منطقه 2 - 9809         148
..   ...   ...   ...                         ...         ...
73    15    01    01        ناحیه 1 کمربند شمالی           2
74    16  10.2    01   ناحیه 10.2 سازمان پارک‌ها           4
75    16  11.2    01   ناحیه 11.2 سازمان پارک‌ها           4
76    16  12.2    01   ناحیه 12.2 سازمان پارک‌ها           2
77    16   7.1    01    ناحیه 7.1 سازمان پارک‌ها          18

[78 rows x 5 columns]
     منطقه ناحیه پیمان  لکه  \
0       01    01    01  001   
1       01    01   

/home/pooya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:695: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [3]:
#  ETL Calculation

del tmpG, tmp

ETP = {
    'Dry': [122.6, 176.6, 220.8, 255.8, 236.9, 185.8, 123.7, 64.2, 41.9, 35.3, 42.0, 79.9],
    'Normal': [116.1, 168.9, 206.6, 231.3, 201.4, 172, 104.2, 56.9, 38, 39.8, 49.6, 81.1],
    'Wet': [103.4, 151.3, 196.2, 219.3, 199.0, 157.9, 109.0, 63.8, 38.7, 27.8, 32.7, 60.7]
}

Prec = {
    'Dry': [19.6, 14.6, 5.8, 0.5, 0.0, 0.1, 2.6, 7.4, 11.4, 21.1, 21.3, 23.2],
    'Normal': [22.9, 16.3, 5.5, 0.1, 0.0, 0.0, 6.0, 4.3, 3.5, 3.4, 2.2, 30.1],
    'Wet': [30.4, 19.5, 5.3, 0.5, 0.1, 0.3, 0.6, 3.6, 14.1, 23.5, 25.7, 28.6]
}

for i in ['Dry', 'Normal', 'Wet']:

    ETL_Info = {
        'Number_Day': [31, 31, 31, 31, 31, 31, 30, 30, 30, 30, 30, 29],
        'PET': ETP[i],
        'Eff_Precipitation': Prec[i],
        'Ground_Water': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        'Deficit_Irrigation': [0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95],
        'LF': [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05],
        'Shading_Area': [1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00],
        'Percentage_Wetted_Area': [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]
    }

    Month_Name = ['M01', 'M02', 'M03', 'M04', 'M05', 'M06',
                  'M07', 'M08', 'M09', 'M10', 'M11', 'M12']

    ETL_Info = pd.DataFrame(ETL_Info, index=Month_Name)

    tmpG = data.groupby(
        by=['Region', 'District', 'Peyman', 'Address', 'Irrigation']
    )

    tmpG = tmpG['KL_M01', 'KL_M02', 'KL_M03', 'KL_M04', 'KL_M05', 'KL_M06',
                'KL_M07', 'KL_M08', 'KL_M09', 'KL_M10', 'KL_M11', 'KL_M12',
                'SubIrrigation_Area', 'Eff']

    tmp = tmpG.mean().reset_index()

    month = "M01"
    tmp = tmp >> define(ETL_M01='(KL_M01 * ETL_Info.loc[month]["PET"] * ETL_Info.loc[month]["Deficit_Irrigation"] * ETL_Info.loc[month]["Shading_Area"] - (ETL_Info.loc[month]["Eff_Precipitation"] + ETL_Info.loc[month]["Ground_Water"]) * ETL_Info.loc[month]["Percentage_Wetted_Area"] / 100) / ETL_Info.loc[month]["Number_Day"] / (1 - ETL_Info.loc[month]["LF"]) / Eff')
    tmp = tmp >> define(ETL_M01=if_else('ETL_M01 < 0', 0, 'ETL_M01'))
    tmp = tmp >> define(
        IrriReq_M01='ETL_M01 * ETL_Info.loc[month]["Number_Day"] * SubIrrigation_Area / 1000 / 1000'
    )

    month = "M02"
    tmp = tmp >> define(ETL_M02='(KL_M02 * ETL_Info.loc[month]["PET"] * ETL_Info.loc[month]["Deficit_Irrigation"] * ETL_Info.loc[month]["Shading_Area"] - (ETL_Info.loc[month]["Eff_Precipitation"] + ETL_Info.loc[month]["Ground_Water"]) * ETL_Info.loc[month]["Percentage_Wetted_Area"] / 100) / ETL_Info.loc[month]["Number_Day"] / (1 - ETL_Info.loc[month]["LF"]) / Eff')
    tmp = tmp >> define(ETL_M02=if_else('ETL_M02 < 0', 0, 'ETL_M02'))
    tmp = tmp >> define(
        IrriReq_M02='ETL_M02 * ETL_Info.loc[month]["Number_Day"] * SubIrrigation_Area / 1000 / 1000'
    )

    month = "M03"
    tmp = tmp >> define(ETL_M03='(KL_M03 * ETL_Info.loc[month]["PET"] * ETL_Info.loc[month]["Deficit_Irrigation"] * ETL_Info.loc[month]["Shading_Area"] - (ETL_Info.loc[month]["Eff_Precipitation"] + ETL_Info.loc[month]["Ground_Water"]) * ETL_Info.loc[month]["Percentage_Wetted_Area"] / 100) / ETL_Info.loc[month]["Number_Day"] / (1 - ETL_Info.loc[month]["LF"]) / Eff')
    tmp = tmp >> define(ETL_M03=if_else('ETL_M03 < 0', 0, 'ETL_M03'))
    tmp = tmp >> define(
        IrriReq_M03='ETL_M03 * ETL_Info.loc[month]["Number_Day"] * SubIrrigation_Area / 1000 / 1000'
    )

    month = "M04"
    tmp = tmp >> define(ETL_M04='(KL_M04 * ETL_Info.loc[month]["PET"] * ETL_Info.loc[month]["Deficit_Irrigation"] * ETL_Info.loc[month]["Shading_Area"] - (ETL_Info.loc[month]["Eff_Precipitation"] + ETL_Info.loc[month]["Ground_Water"]) * ETL_Info.loc[month]["Percentage_Wetted_Area"] / 100) / ETL_Info.loc[month]["Number_Day"] / (1 - ETL_Info.loc[month]["LF"]) / Eff')
    tmp = tmp >> define(ETL_M04=if_else('ETL_M04 < 0', 0, 'ETL_M04'))
    tmp = tmp >> define(
        IrriReq_M04='ETL_M04 * ETL_Info.loc[month]["Number_Day"] * SubIrrigation_Area / 1000 / 1000'
    )

    month = "M05"
    tmp = tmp >> define(ETL_M05='(KL_M05 * ETL_Info.loc[month]["PET"] * ETL_Info.loc[month]["Deficit_Irrigation"] * ETL_Info.loc[month]["Shading_Area"] - (ETL_Info.loc[month]["Eff_Precipitation"] + ETL_Info.loc[month]["Ground_Water"]) * ETL_Info.loc[month]["Percentage_Wetted_Area"] / 100) / ETL_Info.loc[month]["Number_Day"] / (1 - ETL_Info.loc[month]["LF"]) / Eff')
    tmp = tmp >> define(ETL_M05=if_else('ETL_M05 < 0', 0, 'ETL_M05'))
    tmp = tmp >> define(
        IrriReq_M05='ETL_M05 * ETL_Info.loc[month]["Number_Day"] * SubIrrigation_Area / 1000 / 1000'
    )

    month = "M06"
    tmp = tmp >> define(ETL_M06='(KL_M06 * ETL_Info.loc[month]["PET"] * ETL_Info.loc[month]["Deficit_Irrigation"] * ETL_Info.loc[month]["Shading_Area"] - (ETL_Info.loc[month]["Eff_Precipitation"] + ETL_Info.loc[month]["Ground_Water"]) * ETL_Info.loc[month]["Percentage_Wetted_Area"] / 100) / ETL_Info.loc[month]["Number_Day"] / (1 - ETL_Info.loc[month]["LF"]) / Eff')
    tmp = tmp >> define(ETL_M06=if_else('ETL_M06 < 0', 0, 'ETL_M06'))
    tmp = tmp >> define(
        IrriReq_M06='ETL_M06 * ETL_Info.loc[month]["Number_Day"] * SubIrrigation_Area / 1000 / 1000'
    )

    month = "M07"
    tmp = tmp >> define(ETL_M07='(KL_M07 * ETL_Info.loc[month]["PET"] * ETL_Info.loc[month]["Deficit_Irrigation"] * ETL_Info.loc[month]["Shading_Area"] - (ETL_Info.loc[month]["Eff_Precipitation"] + ETL_Info.loc[month]["Ground_Water"]) * ETL_Info.loc[month]["Percentage_Wetted_Area"] / 100) / ETL_Info.loc[month]["Number_Day"] / (1 - ETL_Info.loc[month]["LF"]) / Eff')
    tmp = tmp >> define(ETL_M07=if_else('ETL_M07 < 0', 0, 'ETL_M07'))
    tmp = tmp >> define(
        IrriReq_M07='ETL_M07 * ETL_Info.loc[month]["Number_Day"] * SubIrrigation_Area / 1000 / 1000'
    )

    month = "M08"
    tmp = tmp >> define(ETL_M08='(KL_M08 * ETL_Info.loc[month]["PET"] * ETL_Info.loc[month]["Deficit_Irrigation"] * ETL_Info.loc[month]["Shading_Area"] - (ETL_Info.loc[month]["Eff_Precipitation"] + ETL_Info.loc[month]["Ground_Water"]) * ETL_Info.loc[month]["Percentage_Wetted_Area"] / 100) / ETL_Info.loc[month]["Number_Day"] / (1 - ETL_Info.loc[month]["LF"]) / Eff')
    tmp = tmp >> define(ETL_M08=if_else('ETL_M08 < 0', 0, 'ETL_M08'))
    tmp = tmp >> define(
        IrriReq_M08='ETL_M08 * ETL_Info.loc[month]["Number_Day"] * SubIrrigation_Area / 1000 / 1000'
    )

    month = "M09"
    tmp = tmp >> define(ETL_M09='(KL_M09 * ETL_Info.loc[month]["PET"] * ETL_Info.loc[month]["Deficit_Irrigation"] * ETL_Info.loc[month]["Shading_Area"] - (ETL_Info.loc[month]["Eff_Precipitation"] + ETL_Info.loc[month]["Ground_Water"]) * ETL_Info.loc[month]["Percentage_Wetted_Area"] / 100) / ETL_Info.loc[month]["Number_Day"] / (1 - ETL_Info.loc[month]["LF"]) / Eff')
    tmp = tmp >> define(ETL_M09=if_else('ETL_M09 < 0', 0, 'ETL_M09'))
    tmp = tmp >> define(
        IrriReq_M09='ETL_M09 * ETL_Info.loc[month]["Number_Day"] * SubIrrigation_Area / 1000 / 1000'
    )

    month = "M10"
    tmp = tmp >> define(ETL_M10='(KL_M10 * ETL_Info.loc[month]["PET"] * ETL_Info.loc[month]["Deficit_Irrigation"] * ETL_Info.loc[month]["Shading_Area"] - (ETL_Info.loc[month]["Eff_Precipitation"] + ETL_Info.loc[month]["Ground_Water"]) * ETL_Info.loc[month]["Percentage_Wetted_Area"] / 100) / ETL_Info.loc[month]["Number_Day"] / (1 - ETL_Info.loc[month]["LF"]) / Eff')
    tmp = tmp >> define(ETL_M10=if_else('ETL_M10 < 0', 0, 'ETL_M10'))
    tmp = tmp >> define(
        IrriReq_M10='ETL_M10 * ETL_Info.loc[month]["Number_Day"] * SubIrrigation_Area / 1000 / 1000'
    )

    month = "M11"
    tmp = tmp >> define(ETL_M11='(KL_M11 * ETL_Info.loc[month]["PET"] * ETL_Info.loc[month]["Deficit_Irrigation"] * ETL_Info.loc[month]["Shading_Area"] - (ETL_Info.loc[month]["Eff_Precipitation"] + ETL_Info.loc[month]["Ground_Water"]) * ETL_Info.loc[month]["Percentage_Wetted_Area"] / 100) / ETL_Info.loc[month]["Number_Day"] / (1 - ETL_Info.loc[month]["LF"]) / Eff')
    tmp = tmp >> define(ETL_M11=if_else('ETL_M11 < 0', 0, 'ETL_M11'))
    tmp = tmp >> define(
        IrriReq_M11='ETL_M11 * ETL_Info.loc[month]["Number_Day"] * SubIrrigation_Area / 1000 / 1000'
    )

    month = "M12"
    tmp = tmp >> define(ETL_M12='(KL_M12 * ETL_Info.loc[month]["PET"] * ETL_Info.loc[month]["Deficit_Irrigation"] * ETL_Info.loc[month]["Shading_Area"] - (ETL_Info.loc[month]["Eff_Precipitation"] + ETL_Info.loc[month]["Ground_Water"]) * ETL_Info.loc[month]["Percentage_Wetted_Area"] / 100) / ETL_Info.loc[month]["Number_Day"] / (1 - ETL_Info.loc[month]["LF"]) / Eff')
    tmp = tmp >> define(ETL_M12=if_else('ETL_M12 < 0', 0, 'ETL_M12'))
    tmp = tmp >> define(
        IrriReq_M12='ETL_M12 * ETL_Info.loc[month]["Number_Day"] * SubIrrigation_Area / 1000 / 1000'
    )

    tmp = tmp >> define(ETL_YEAR='ETL_M01 * ETL_Info.loc["M01"]["Number_Day"] + ETL_M02 * ETL_Info.loc["M02"]["Number_Day"] + ETL_M03 * ETL_Info.loc["M03"]["Number_Day"] + ETL_M04 * ETL_Info.loc["M04"]["Number_Day"] + ETL_M05 * ETL_Info.loc["M05"]["Number_Day"] + ETL_M06 * ETL_Info.loc["M06"]["Number_Day"] + ETL_M07 * ETL_Info.loc["M07"]["Number_Day"] + ETL_M08 * ETL_Info.loc["M08"]["Number_Day"] + ETL_M09 * ETL_Info.loc["M09"]["Number_Day"] + ETL_M10 * ETL_Info.loc["M10"]["Number_Day"] + ETL_M11 * ETL_Info.loc["M11"]["Number_Day"] + ETL_M12 * ETL_Info.loc["M12"]["Number_Day"]')
    tmp = tmp >> define(
        IrriReq_YEAR='ETL_YEAR * SubIrrigation_Area / 1000 / 1000'
    )

    tmp = tmp >> select(
        'Region', 'District', 'Peyman', 'Address', 'Irrigation', 'SubIrrigation_Area',
        'ETL_M01', 'ETL_M02', 'ETL_M03',
        'ETL_M04', 'ETL_M05', 'ETL_M06',
        'ETL_M07', 'ETL_M08', 'ETL_M09',
        'ETL_M10', 'ETL_M11', 'ETL_M12',
        'IrriReq_M01', 'IrriReq_M02', 'IrriReq_M03',
        'IrriReq_M04', 'IrriReq_M05', 'IrriReq_M06',
        'IrriReq_M07', 'IrriReq_M08', 'IrriReq_M09',
        'IrriReq_M10', 'IrriReq_M11', 'IrriReq_M12',
        'ETL_YEAR', 'IrriReq_YEAR'
    )

    tmp.columns = ['Region', 'District', 'Peyman', 'Address',
                   'Irrigation', 'SubIrrigation_Area'] + list(i + '_' + tmp.columns[6:])

    try:
        final_result
    except NameError:
        final_result = tmp
    else:
        tmp = tmp >> select('-SubIrrigation_Area')
        final_result = pd.merge(final_result,
                                tmp,
                                how='left',
                                on=['Region', 'District', 'Peyman', 'Address', 'Irrigation'])

final_result.to_excel(PROJECT_PATH + "Report/Final_Result.xlsx",
                      index=False)

/home/pooya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [4]:
final_result

,Region,District,Peyman,Address,Irrigation,SubIrrigation_Area,Dry_ETL_M01,Dry_ETL_M02,Dry_ETL_M03,Dry_ETL_M04,...,Wet_IrriReq_M05,Wet_IrriReq_M06,Wet_IrriReq_M07,Wet_IrriReq_M08,Wet_IrriReq_M09,Wet_IrriReq_M10,Wet_IrriReq_M11,Wet_IrriReq_M12,Wet_ETL_YEAR,Wet_IrriReq_YEAR
0,01,01,01,001,02,1405.0,2.371033,5.440192,9.073566,12.074069,...,0.389323,0.259564,0.133289,0.050288,0.0,0.0,0.0,0.0,1332.827193,1.872622
1,01,01,01,001,03,1313.0,2.274147,4.972997,8.144758,10.779914,...,0.324744,0.216586,0.111365,0.043124,0.0,0.0,0.0,0.0,1204.596563,1.581635
2,01,01,01,002,03,1568.0,1.452001,3.492664,5.923756,7.920955,...,0.285108,0.190023,0.097464,0.035902,0.0,0.0,0.0,0.0,864.750082,1.355928
3,01,01,01,002,04,14686.0,2.229887,4.893303,8.025190,10.626002,...,3.580498,2.387922,1.227713,0.474479,0.0,0.0,0.0,0.0,1186.300891,17.422015
4,01,01,01,003,03,3168.0,2.278227,4.980344,8.155780,10.794103,...,0.784570,0.523265,0.269058,0.104205,0.0,0.0,0.0,0.0,1206.283179,3.821505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2833,16,7.1,01,001,02,54487.0,1.382324,3.659950,6.402599,8.635897,...,10.806247,7.198260,3.684268,1.298409,0.0,0.0,0.0,0.0,924.129276,50.353032
2834,16,7.1,01,001,03,26623.0,1.733312,3.999185,6.683708,8.899195,...,5.437513,3.625085,1.861248,0.700189,0.0,0.0,0.0,0.0,981.034134,26.118072
2835,16,7.1,01,002,01,23000.0,0.982931,2.773533,4.942467,6.700042,...,3.539878,2.357222,1.205031,0.413596,0.0,0.0,0.0,0.0,708.645047,16.298836
2836,16,7.1,01,002,03,109792.0,1.397738,3.394960,5.777166,7.732259,...,19.488727,12.988350,6.660254,2.441774,0.0,0.0,0.0,0.0,842.319642,92.479958


In [5]:
#  Load Shapefiles
region_shp = PROJECT_PATH + 'Data/Mashhad_City_Layers/Region/ShapeFile/Regions.shp'
district_shp = PROJECT_PATH + \
    'Data/Mashhad_City_Layers/District/ShapeFile/Districts.shp'

region_df = gpd.read_file(filename=region_shp)[['Name', 'geometry']]
district_df = gpd.read_file(filename=district_shp)[['Name', 'geometry']]

region_df.columns = ['ID', 'geometry']
district_df.columns = ['ID', 'geometry']

# Region - Dry

In [7]:
#  Report And Plot ETL
tmpData = final_result >> select(
    'Region', 'District', 'Peyman', 'Address', 'Irrigation', 'SubIrrigation_Area',
    endswith='_YEAR'
)

tmpG = tmpData.groupby(['Region'])

tmp = tmpG.agg({'SubIrrigation_Area': 'sum',
                'Dry_ETL_YEAR': 'mean',
                'Normal_ETL_YEAR': 'mean',
                'Wet_ETL_YEAR': 'mean',
                'Dry_IrriReq_YEAR': 'sum',
                'Normal_IrriReq_YEAR': 'sum',
                'Wet_IrriReq_YEAR': 'sum'})

tmp = tmp.reset_index().round(0)

file_name = PROJECT_PATH + "Report/Report_ETL_Region.xlsx"
tmp.to_excel(file_name, index=False)

tmp['ID'] = 'Region' + ' ' + tmp['Region']

tmp = region_df >> left_join(tmp, on='ID')

# Define function that returns json_data for year selected by user.    
def json_data(data, climate):    
    df = data >> select('ID', 'geometry', 'Region', 'SubIrrigation_Area', startswith=climate)
    df.columns = ['ID', 'geometry', 'Region', 'Area', 'ETL', 'IrriReq']
    print(df[['Region', 'IrriReq']])
    df_json = json.loads(df.to_json())
    df_dump = json.dumps(df_json)
    return df_dump

# Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson=json_data(tmp, 'Dry'))

# Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][9]

# Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors. Input nan_color.
color_mapper = LinearColorMapper(palette=palette,
                                 low=0,
                                 high=1800)

tick_labels = {'0': '0',
               '200': '200',
               '400': '400',
               '600': '600',
               '800': '800',
               '1000': '1000',
               '1200': '1200',
               '1400': '1400',
               '1600': '1600',
               '1800': '1800'}

hover = HoverTool(tooltips=[('منطقه', '@Region'),
                            ('مساحت تحت آبیاری بر حسب متر مربع',
                             '@Area'),
                            ('نیاز آبیاری بر حسب میلیمتر در سال', '@ETL'),
                            ('حجم آبیاری بر حسب میلیون متر مکعب', '@IrriReq')])

color_bar = ColorBar(color_mapper=color_mapper,
                     label_standoff=8,
                     width=400,
                     height=20,
                     border_line_color=None,
                     location=(0, 0),
                     orientation='horizontal',
                     major_label_overrides=tick_labels)

p = figure(title='Mashhad',
           toolbar_location=None,
           tools=[hover])

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

p.patches('xs',
          'ys',
          source=geosource,
          fill_color={'field': 'IrriReq',
                      'transform': color_mapper},
          line_color='black',
          line_width=0.25,
          fill_alpha=1)

p.add_layout(color_bar,
             'below')

# # Define the callback function: update_plot
# def update_plot(attr, old, new):
#     yr = slider.value
    
#     n = {1 : 'Dry', 2 : 'Normal', 3 : 'Wet'}
    
#     new_data = json_data(n[yr])
#     geosource.geojson = new_data
#     p.title.text = f'Mashhad: Climate = {n[yr]}' 
    
# # Make a slider object: slider 
# slider = Slider(title = 'Climate',start = 1, end = 3, step = 1, value = 2)
# slider.on_change('value', update_plot)

# # Make a column layout of widgetbox(slider) and plot, and add it to the current document
# layout = column(p, widgetbox(slider))
# curdoc().add_root(layout)

#Display plot inline in Jupyter notebook
output_notebook()

#Display plot
show(p)

   Region  IrriReq
0      01    490.0
1      02   1113.0
2      03    890.0
3      03    890.0
4      04    470.0
5      05    476.0
6      06    753.0
7      06    753.0
8      07   2076.0
9      08    547.0
10     09   1665.0
11     10   1322.0
12     11   1047.0
13     12    528.0
14     12    528.0
15     13     87.0


Loading BokehJS ...

# Region - Normal

In [8]:
#  Report And Plot ETL
tmpData = final_result >> select(
    'Region', 'District', 'Peyman', 'Address', 'Irrigation', 'SubIrrigation_Area',
    endswith='_YEAR'
)

tmpG = tmpData.groupby(['Region'])

tmp = tmpG.agg({'SubIrrigation_Area': 'sum',
                'Dry_ETL_YEAR': 'mean',
                'Normal_ETL_YEAR': 'mean',
                'Wet_ETL_YEAR': 'mean',
                'Dry_IrriReq_YEAR': 'sum',
                'Normal_IrriReq_YEAR': 'sum',
                'Wet_IrriReq_YEAR': 'sum'})

tmp = tmp.reset_index().round(0)

file_name = PROJECT_PATH + "Report/Report_ETL_Region.xlsx"
tmp.to_excel(file_name, index=False)

tmp['ID'] = 'Region' + ' ' + tmp['Region']

tmp = region_df >> left_join(tmp, on='ID')

# Define function that returns json_data for year selected by user.    
def json_data(data, climate):    
    df = data >> select('ID', 'geometry', 'Region', 'SubIrrigation_Area', startswith=climate)
    df.columns = ['ID', 'geometry', 'Region', 'Area', 'ETL', 'IrriReq']
    print(df[['Region', 'IrriReq']])
    df_json = json.loads(df.to_json())
    df_dump = json.dumps(df_json)
    return df_dump

# Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson=json_data(tmp, 'Normal'))

# Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][9]

# Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors. Input nan_color.
color_mapper = LinearColorMapper(palette=palette,
                                 low=0,
                                 high=2000)

tick_labels = {'0': '0',
               '200': '200',
               '400': '400',
               '600': '600',
               '800': '800',
               '1000': '1000',
               '1200': '1200',
               '1400': '1400',
               '1600': '1600',
               '1800': '> 1800'}

hover = HoverTool(tooltips=[('منطقه', '@Region'),
                            ('مساحت تحت آبیاری بر حسب متر مربع',
                             '@Area'),
                            ('نیاز آبیاری بر حسب میلیمتر در سال', '@ETL'),
                            ('حجم آبیاری بر حسب میلیون متر مکعب', '@IrriReq')])

color_bar = ColorBar(color_mapper=color_mapper,
                     label_standoff=8,
                     width=500,
                     height=20,
                     border_line_color=None,
                     location=(0, 0),
                     orientation='horizontal',
                     major_label_overrides=tick_labels)

p = figure(title='Mashhad',
           toolbar_location=None,
           tools=[hover])

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

p.patches('xs',
          'ys',
          source=geosource,
          fill_color={'field': 'IrriReq',
                      'transform': color_mapper},
          line_color='black',
          line_width=0.25,
          fill_alpha=1)

p.add_layout(color_bar,
             'below')

# # Define the callback function: update_plot
# def update_plot(attr, old, new):
#     yr = slider.value
    
#     n = {1 : 'Dry', 2 : 'Normal', 3 : 'Wet'}
    
#     new_data = json_data(n[yr])
#     geosource.geojson = new_data
#     p.title.text = f'Mashhad: Climate = {n[yr]}' 
    
# # Make a slider object: slider 
# slider = Slider(title = 'Climate',start = 1, end = 3, step = 1, value = 2)
# slider.on_change('value', update_plot)

# # Make a column layout of widgetbox(slider) and plot, and add it to the current document
# layout = column(p, widgetbox(slider))
# curdoc().add_root(layout)

#Display plot inline in Jupyter notebook
output_notebook()

#Display plot
show(p)

   Region  IrriReq
0      01    449.0
1      02   1020.0
2      03    816.0
3      03    816.0
4      04    431.0
5      05    436.0
6      06    690.0
7      06    690.0
8      07   1898.0
9      08    502.0
10     09   1525.0
11     10   1211.0
12     11    960.0
13     12    484.0
14     12    484.0
15     13     79.0


Loading BokehJS ...

# Region - Wet

In [9]:
#  Report And Plot ETL
tmpData = final_result >> select(
    'Region', 'District', 'Peyman', 'Address', 'Irrigation', 'SubIrrigation_Area',
    endswith='_YEAR'
)

tmpG = tmpData.groupby(['Region'])

tmp = tmpG.agg({'SubIrrigation_Area': 'sum',
                'Dry_ETL_YEAR': 'mean',
                'Normal_ETL_YEAR': 'mean',
                'Wet_ETL_YEAR': 'mean',
                'Dry_IrriReq_YEAR': 'sum',
                'Normal_IrriReq_YEAR': 'sum',
                'Wet_IrriReq_YEAR': 'sum'})

tmp = tmp.reset_index().round(0)

file_name = PROJECT_PATH + "Report/Report_ETL_Region.xlsx"
tmp.to_excel(file_name, index=False)

tmp['ID'] = 'Region' + ' ' + tmp['Region']

tmp = region_df >> left_join(tmp, on='ID')

# Define function that returns json_data for year selected by user.    
def json_data(data, climate):    
    df = data >> select('ID', 'geometry', 'Region', 'SubIrrigation_Area', startswith=climate)
    df.columns = ['ID', 'geometry', 'Region', 'Area', 'ETL', 'IrriReq']
    print(df[['Region', 'IrriReq']])
    df_json = json.loads(df.to_json())
    df_dump = json.dumps(df_json)
    return df_dump

# Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson=json_data(tmp, 'Wet'))

# Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][7]

# Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors. Input nan_color.
color_mapper = LinearColorMapper(palette=palette,
                                 low=0,
                                 high=2100)

tick_labels = {'0': '0',
               '300': '300',
               '600': '600',
               '900': '900',
               '1200': '1200',
               '1500': '1500',
               '1800': '1800',
               '2100': '> 2100'}

hover = HoverTool(tooltips=[('منطقه', '@Region'),
                            ('مساحت تحت آبیاری بر حسب متر مربع',
                             '@Area'),
                            ('نیاز آبیاری بر حسب میلیمتر در سال', '@ETL'),
                            ('حجم آبیاری بر حسب میلیون متر مکعب', '@IrriReq')])

color_bar = ColorBar(color_mapper=color_mapper,
                     label_standoff=10,
                     width=350,
                     height=20,
                     border_line_color=None,
                     location=(0, 0),
                     orientation='horizontal',
                     major_label_overrides=tick_labels)

p = figure(title='Mashhad',
           toolbar_location=None,
           tools=[hover])

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

p.patches('xs',
          'ys',
          source=geosource,
          fill_color={'field': 'IrriReq',
                      'transform': color_mapper},
          line_color='black',
          line_width=0.25,
          fill_alpha=1)

p.add_layout(color_bar,
             'below')

# # Define the callback function: update_plot
# def update_plot(attr, old, new):
#     yr = slider.value
    
#     n = {1 : 'Dry', 2 : 'Normal', 3 : 'Wet'}
    
#     new_data = json_data(n[yr])
#     geosource.geojson = new_data
#     p.title.text = f'Mashhad: Climate = {n[yr]}' 
    
# # Make a slider object: slider 
# slider = Slider(title = 'Climate',start = 1, end = 3, step = 1, value = 2)
# slider.on_change('value', update_plot)

# # Make a column layout of widgetbox(slider) and plot, and add it to the current document
# layout = column(p, widgetbox(slider))
# curdoc().add_root(layout)

#Display plot inline in Jupyter notebook
output_notebook()

#Display plot
show(p)

   Region  IrriReq
0      01    411.0
1      02    934.0
2      03    747.0
3      03    747.0
4      04    394.0
5      05    399.0
6      06    632.0
7      06    632.0
8      07   1738.0
9      08    460.0
10     09   1396.0
11     10   1109.0
12     11    879.0
13     12    443.0
14     12    443.0
15     13     73.0


Loading BokehJS ...

# Region - District - Dry

In [10]:
#  Report And Plot ETL
tmpData = final_result >> select(
    'Region', 'District', 'Peyman', 'Address', 'Irrigation', 'SubIrrigation_Area',
    endswith='_YEAR'
)

tmpG = tmpData.groupby(['Region', 'District'])

tmp = tmpG.agg({'SubIrrigation_Area': 'sum',
                'Dry_ETL_YEAR': 'mean',
                'Normal_ETL_YEAR': 'mean',
                'Wet_ETL_YEAR': 'mean',
                'Dry_IrriReq_YEAR': 'sum',
                'Normal_IrriReq_YEAR': 'sum',
                'Wet_IrriReq_YEAR': 'sum'})

tmp = tmp.reset_index().round(0)

file_name = PROJECT_PATH + "Report/Report_ETL_Region_District.xlsx"
tmp.to_excel(file_name, index=False)

tmp['ID'] = 'R' + tmp['Region'] + ' ' + 'D' + tmp['District']

tmp = district_df >> left_join(tmp, on='ID')

# Define function that returns json_data for year selected by user.    
def json_data(data, climate):    
    df = data >> select('ID', 'geometry', 'Region', 'District', 'SubIrrigation_Area', startswith=climate)
    df.columns = ['ID', 'geometry', 'Region', 'District', 'Area', 'ETL', 'IrriReq']
    print(df[['Region', 'District', 'IrriReq']])
    df_json = json.loads(df.to_json())
    df_dump = json.dumps(df_json)
    return df_dump

# Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson=json_data(tmp, 'Dry'))

# Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][7]

# Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors. Input nan_color.
color_mapper = LinearColorMapper(palette=palette,
                                 low=0,
                                 high=1400)

tick_labels = {'0': '0',
               '200': '200',
               '400': '400',
               '600': '600',
               '800': '800',
               '1000': '1000',
               '1200': '1200',
               '1400': '1400'}

hover = HoverTool(tooltips=[('منطقه', '@Region'),
                            ('ناحیه', '@District'),
                            ('مساحت تحت آبیاری بر حسب متر مربع',
                             '@Area'),
                            ('نیاز آبیاری بر حسب میلیمتر در سال', '@ETL'),
                            ('حجم آبیاری بر حسب میلیون متر مکعب', '@IrriReq')])

color_bar = ColorBar(color_mapper=color_mapper,
                     label_standoff=8,
                     width=400,
                     height=20,
                     border_line_color=None,
                     location=(0, 0),
                     orientation='horizontal',
                     major_label_overrides=tick_labels)

p = figure(title='Mashhad',
           toolbar_location=None,
           tools=[hover])

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

p.patches('xs',
          'ys',
          source=geosource,
          fill_color={'field': 'IrriReq',
                      'transform': color_mapper},
          line_color='black',
          line_width=0.25,
          fill_alpha=1)

p.add_layout(color_bar,
             'below')

# # Define the callback function: update_plot
# def update_plot(attr, old, new):
#     yr = slider.value
    
#     n = {1 : 'Dry', 2 : 'Normal', 3 : 'Wet'}
    
#     new_data = json_data(n[yr])
#     geosource.geojson = new_data
#     p.title.text = f'Mashhad: Climate = {n[yr]}' 
    
# # Make a slider object: slider 
# slider = Slider(title = 'Climate',start = 1, end = 3, step = 1, value = 2)
# slider.on_change('value', update_plot)

# # Make a column layout of widgetbox(slider) and plot, and add it to the current document
# layout = column(p, widgetbox(slider))
# curdoc().add_root(layout)

#Display plot inline in Jupyter notebook
output_notebook()

#Display plot
show(p)

   Region District  IrriReq
0      01       01    133.0
1     NaN      NaN      NaN
2      04       02    182.0
3      12       01    177.0
4      12       03    103.0
5      09       01    654.0
6      05       02    200.0
7      01       02    273.0
8      08       01     65.0
9      08       03    172.0
10     07       02   1568.0
11     09       02    381.0
12     03       02    198.0
13     02       04    353.0
14     07       01    374.0
15     06       03    113.0
16     08       02    311.0
17     02       03    128.0
18     06       01     99.0
19    NaN      NaN      NaN
20     04       01    124.0
21     03       03    519.0
22     11       01    722.0
23     03       04     44.0
24     13       01     39.0
25     07       03    134.0
26     09       03    629.0
27     10       03    348.0
28     13       02     47.0
29     02       01    380.0
30     02       02    253.0
31     06       03    113.0
32     04       03    164.0
33     10       01    598.0
34     03       01  

Loading BokehJS ...

# Region - District - Normal

In [11]:
#  Report And Plot ETL
tmpData = final_result >> select(
    'Region', 'District', 'Peyman', 'Address', 'Irrigation', 'SubIrrigation_Area',
    endswith='_YEAR'
)

tmpG = tmpData.groupby(['Region', 'District'])

tmp = tmpG.agg({'SubIrrigation_Area': 'sum',
                'Dry_ETL_YEAR': 'mean',
                'Normal_ETL_YEAR': 'mean',
                'Wet_ETL_YEAR': 'mean',
                'Dry_IrriReq_YEAR': 'sum',
                'Normal_IrriReq_YEAR': 'sum',
                'Wet_IrriReq_YEAR': 'sum'})

tmp = tmp.reset_index().round(0)

file_name = PROJECT_PATH + "Report/Report_ETL_Region_District.xlsx"
tmp.to_excel(file_name, index=False)

tmp['ID'] = 'R' + tmp['Region'] + ' ' + 'D' + tmp['District']

tmp = district_df >> left_join(tmp, on='ID')

# Define function that returns json_data for year selected by user.    
def json_data(data, climate):    
    df = data >> select('ID', 'geometry', 'Region', 'District', 'SubIrrigation_Area', startswith=climate)
    df.columns = ['ID', 'geometry', 'Region', 'District', 'Area', 'ETL', 'IrriReq']
    print(df[['Region', 'District', 'IrriReq']])
    df_json = json.loads(df.to_json())
    df_dump = json.dumps(df_json)
    return df_dump

# Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson=json_data(tmp, 'Normal'))

# Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][7]

# Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors. Input nan_color.
color_mapper = LinearColorMapper(palette=palette,
                                 low=0,
                                 high=1400)

tick_labels = {'0': '0',
               '200': '200',
               '400': '400',
               '600': '600',
               '800': '800',
               '1000': '1000',
               '1200': '1200',
               '1400': '1400'}

hover = HoverTool(tooltips=[('منطقه', '@Region'),
                            ('ناحیه', '@District'),
                            ('مساحت تحت آبیاری بر حسب متر مربع',
                             '@Area'),
                            ('نیاز آبیاری بر حسب میلیمتر در سال', '@ETL'),
                            ('حجم آبیاری بر حسب میلیون متر مکعب', '@IrriReq')])

color_bar = ColorBar(color_mapper=color_mapper,
                     label_standoff=8,
                     width=400,
                     height=20,
                     border_line_color=None,
                     location=(0, 0),
                     orientation='horizontal',
                     major_label_overrides=tick_labels)

p = figure(title='Mashhad',
           toolbar_location=None,
           tools=[hover])

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

p.patches('xs',
          'ys',
          source=geosource,
          fill_color={'field': 'IrriReq',
                      'transform': color_mapper},
          line_color='black',
          line_width=0.25,
          fill_alpha=1)

p.add_layout(color_bar,
             'below')

# # Define the callback function: update_plot
# def update_plot(attr, old, new):
#     yr = slider.value
    
#     n = {1 : 'Dry', 2 : 'Normal', 3 : 'Wet'}
    
#     new_data = json_data(n[yr])
#     geosource.geojson = new_data
#     p.title.text = f'Mashhad: Climate = {n[yr]}' 
    
# # Make a slider object: slider 
# slider = Slider(title = 'Climate',start = 1, end = 3, step = 1, value = 2)
# slider.on_change('value', update_plot)

# # Make a column layout of widgetbox(slider) and plot, and add it to the current document
# layout = column(p, widgetbox(slider))
# curdoc().add_root(layout)

#Display plot inline in Jupyter notebook
output_notebook()

#Display plot
show(p)

   Region District  IrriReq
0      01       01    122.0
1     NaN      NaN      NaN
2      04       02    167.0
3      12       01    162.0
4      12       03     95.0
5      09       01    600.0
6      05       02    183.0
7      01       02    250.0
8      08       01     59.0
9      08       03    157.0
10     07       02   1433.0
11     09       02    349.0
12     03       02    182.0
13     02       04    323.0
14     07       01    343.0
15     06       03    103.0
16     08       02    285.0
17     02       03    117.0
18     06       01     91.0
19    NaN      NaN      NaN
20     04       01    114.0
21     03       03    475.0
22     11       01    662.0
23     03       04     40.0
24     13       01     36.0
25     07       03    123.0
26     09       03    576.0
27     10       03    319.0
28     13       02     43.0
29     02       01    348.0
30     02       02    232.0
31     06       03    103.0
32     04       03    150.0
33     10       01    548.0
34     03       01  

Loading BokehJS ...

# Region - District - Wet

In [12]:
#  Report And Plot ETL
tmpData = final_result >> select(
    'Region', 'District', 'Peyman', 'Address', 'Irrigation', 'SubIrrigation_Area',
    endswith='_YEAR'
)

tmpG = tmpData.groupby(['Region', 'District'])

tmp = tmpG.agg({'SubIrrigation_Area': 'sum',
                'Dry_ETL_YEAR': 'mean',
                'Normal_ETL_YEAR': 'mean',
                'Wet_ETL_YEAR': 'mean',
                'Dry_IrriReq_YEAR': 'sum',
                'Normal_IrriReq_YEAR': 'sum',
                'Wet_IrriReq_YEAR': 'sum'})

tmp = tmp.reset_index().round(0)

file_name = PROJECT_PATH + "Report/Report_ETL_Region_District.xlsx"
tmp.to_excel(file_name, index=False)

tmp['ID'] = 'R' + tmp['Region'] + ' ' + 'D' + tmp['District']

tmp = district_df >> left_join(tmp, on='ID')

# Define function that returns json_data for year selected by user.    
def json_data(data, climate):    
    df = data >> select('ID', 'geometry', 'Region', 'District', 'SubIrrigation_Area', startswith=climate)
    df.columns = ['ID', 'geometry', 'Region', 'District', 'Area', 'ETL', 'IrriReq']
    print(df[['Region', 'District', 'IrriReq']])
    df_json = json.loads(df.to_json())
    df_dump = json.dumps(df_json)
    return df_dump

# Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson=json_data(tmp, 'Wet'))

# Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][7]

# Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors. Input nan_color.
color_mapper = LinearColorMapper(palette=palette,
                                 low=0,
                                 high=1600)

tick_labels = {'0': '0',
               '100': '100',
               '200': '200',
               '300': '300',
               '400': '400',
               '500': '500',
               '600': '600',
               '700': '700',
               '800': '800',
               '900': '> 800'}

hover = HoverTool(tooltips=[('منطقه', '@Region'),
                            ('ناحیه', '@District'),
                            ('مساحت تحت آبیاری بر حسب متر مربع',
                             '@Area'),
                            ('نیاز آبیاری بر حسب میلیمتر در سال', '@ETL'),
                            ('حجم آبیاری بر حسب میلیون متر مکعب', '@IrriReq')])

color_bar = ColorBar(color_mapper=color_mapper,
                     label_standoff=8,
                     width=400,
                     height=20,
                     border_line_color=None,
                     location=(0, 0),
                     orientation='horizontal',
                     major_label_overrides=tick_labels)

p = figure(title='Mashhad',
           toolbar_location=None,
           tools=[hover])

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

p.patches('xs',
          'ys',
          source=geosource,
          fill_color={'field': 'IrriReq',
                      'transform': color_mapper},
          line_color='black',
          line_width=0.25,
          fill_alpha=1)

p.add_layout(color_bar,
             'below')

# # Define the callback function: update_plot
# def update_plot(attr, old, new):
#     yr = slider.value
    
#     n = {1 : 'Dry', 2 : 'Normal', 3 : 'Wet'}
    
#     new_data = json_data(n[yr])
#     geosource.geojson = new_data
#     p.title.text = f'Mashhad: Climate = {n[yr]}' 
    
# # Make a slider object: slider 
# slider = Slider(title = 'Climate',start = 1, end = 3, step = 1, value = 2)
# slider.on_change('value', update_plot)

# # Make a column layout of widgetbox(slider) and plot, and add it to the current document
# layout = column(p, widgetbox(slider))
# curdoc().add_root(layout)

#Display plot inline in Jupyter notebook
output_notebook()

#Display plot
show(p)

   Region District  IrriReq
0      01       01    112.0
1     NaN      NaN      NaN
2      04       02    153.0
3      12       01    148.0
4      12       03     87.0
5      09       01    549.0
6      05       02    168.0
7      01       02    229.0
8      08       01     54.0
9      08       03    144.0
10     07       02   1312.0
11     09       02    320.0
12     03       02    166.0
13     02       04    296.0
14     07       01    313.0
15     06       03     94.0
16     08       02    261.0
17     02       03    107.0
18     06       01     83.0
19    NaN      NaN      NaN
20     04       01    104.0
21     03       03    435.0
22     11       01    606.0
23     03       04     37.0
24     13       01     33.0
25     07       03    112.0
26     09       03    527.0
27     10       03    292.0
28     13       02     40.0
29     02       01    319.0
30     02       02    213.0
31     06       03     94.0
32     04       03    137.0
33     10       01    501.0
34     03       01  

Loading BokehJS ...